In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df

import datetime
from datetime import datetime as dt
import matplotlib.pyplot as plt

import os


#### 6. data cleaning & normalization for [master_df]
#### 7. train-test-split (80-20)
#### 8. train and test models -> get [train_r2, train_rmse, test_r2, test_rmse]

In [4]:
print(os.getcwd())
DATA_FOLDER = '/Users/vbc5136/Documents/Acads/3-Fall23/RA:Thesis/_SRA-2023/_Data/'
RESULTS_FOLDER = '/Users/vbc5136/Documents/Acads/3-Fall23/RA:Thesis/_SRA-2023/_Results/'
CODE_FOLDER = '/Users/vbc5136/Documents/Acads/3-Fall23/RA:Thesis/_SRA-2023/_Code/'

NARR_DATA_FOLDER = DATA_FOLDER + 'NARR/'
NARR_RAW_DATA_FOLDER = NARR_DATA_FOLDER + 'Raw-Data/'
NARR_FINAL_DATA_FOLDER = NARR_DATA_FOLDER + 'Final-Data/'
NARR_FINAL_DATA_DONE_FOLDER = NARR_DATA_FOLDER + 'Final-Data-Done/'

NARR_FINAL_DATA_FILTER_FOLDER = NARR_DATA_FOLDER + 'Final-Data-Filter/'
NARR_FINAL_DATA_MONTHLY_FOLDER = NARR_DATA_FOLDER + 'Final-Data-Monthly-dfs/'

NARR_FINAL_DATA_AGG_FOLDER = NARR_DATA_FOLDER + 'Final-Data-Monthly-Agg/'
NARR_FINAL_DATA_VAR_AGG_FOLDER = NARR_DATA_FOLDER + 'Final-Data-Variable-Agg/'


print(os.getcwd())
DATA_FOLDER = '/Users/vbc5136/Documents/Acads/3-Fall23/RA:Thesis/_SRA-2023/_Data/'
RESULTS_FOLDER = '/Users/vbc5136/Documents/Acads/3-Fall23/RA:Thesis/_SRA-2023/_Results/'
CODE_FOLDER = '/Users/vbc5136/Documents/Acads/3-Fall23/RA:Thesis/_SRA-2023/_Code/'

EIA_DATA_FOLDER = DATA_FOLDER + 'EIA/'
EIA860_DATA_FOLDER = EIA_DATA_FOLDER + 'EIA-Annual-Plant-Level-Capacity-860/'
EIA923_DATA_FOLDER = EIA_DATA_FOLDER + 'EIA-Annual-Monthly-Generation-Plant-Level-923/'

EIA_FILTER_DATA_STORE_PATH = EIA_DATA_FOLDER + 'Final-Data-Filter/'

if not os.path.isdir(EIA_FILTER_DATA_STORE_PATH):
    os.mkdir(EIA_FILTER_DATA_STORE_PATH)


/Users/vbc5136/Documents/Acads/3-Fall23/RA:Thesis/_SRA-2023/_Code
/Users/vbc5136/Documents/Acads/3-Fall23/RA:Thesis/_SRA-2023/_Code


#### 1. read all Final-Data-Variable-Agg folder files
#### 2. combine all in (1) to get [narr_master_df.xlsx]


In [14]:
narr_final_dfs = os.listdir(NARR_FINAL_DATA_VAR_AGG_FOLDER)
print(narr_final_dfs[0:10], len(narr_final_dfs))
if '.DS_Store' in narr_final_dfs or '~$' not in narr_final_dfs or 'narr_' in narr_final_dfs:
    narr_final_dfs.remove('.DS_Store')
# narr_final_dfs = [name for name in narr_final_dirs if '.xlsx' not in name]
narr_final_dfs = np.sort(narr_final_dfs)
print(narr_final_dfs[0:10], len(narr_final_dfs))

var_names = [file.split('.')[0] for file in narr_final_dfs if (
    '.DS_Store' not in file and '~$' not in file)]
print(var_names, len(var_names))

['~$uwnd.agg.2011.2022.xlsx', 'vwnd.agg.2011.2022.xlsx', '.DS_Store', 'acpcp.agg.2011.2022.xlsx', 'pottmp.agg.2011.2022.xlsx', 'apcp.agg.2011.2022.xlsx', 'albedo.agg.2011.2022.xlsx', 'uwnd.agg.2011.2022.xlsx', '~$vwnd.agg.2011.2022.xlsx', 'tcdc.agg.2011.2022.xlsx'] 13
['acpcp.agg.2011.2022.xlsx' 'air.agg.2011.2022.xlsx'
 'albedo.agg.2011.2022.xlsx' 'apcp.agg.2011.2022.xlsx'
 'dpt.agg.2011.2022.xlsx' 'pottmp.agg.2011.2022.xlsx'
 'rhum.agg.2011.2022.xlsx' 'tcdc.agg.2011.2022.xlsx'
 'uwnd.agg.2011.2022.xlsx' 'vwnd.agg.2011.2022.xlsx'] 12
['acpcp', 'air', 'albedo', 'apcp', 'dpt', 'pottmp', 'rhum', 'tcdc', 'uwnd', 'vwnd'] 10


In [168]:
AIR_TEMP = 'air' # 1.Daily surface air temp. (K)
PRECIP_AMT = 'apcp' # 2.Daily accumulated total surface precip. (kg/m2)
CONV_PRECIP_AMT = 'acpcp' # 3.Daily accumulated conv. surface precip. (kg/m2)
TOTAL_CLOUD_COVER = 'tcdc' # 4.Daily forecast total cloud cover. (%)
UWND = 'uwnd' # 5.Daily u-wind eastward. (m/s)
DEW_POINT_TEMP = 'dpt' # 6.Daily dew point temp. (K)
ALBEDO = 'albedo' # 7.Daily surface albedo. (%)
POT_TEMP = 'pottmp' # 8.Daily potential surface temp. (K)
VWND = 'vwnd' # 9.Daily v-wind nothward. (m/s)
RH = 'rhum' # 10.Daily RH. (%)

PLANT_CODE = 'plant_code'
LATITUDE = 'lat'
LONGITUDE = 'lon'
MONTH = 'month'
YEAR = 'year'
DAY = 'day'

MAX = '_max'
MIN = '_min'
AVG = '_avg'

NAMEPLATE_CAPACITY = 'nameplate_capacity_mw'
ENERGY_SOURCE = 'energy_source_1'
GENERATION = 'net_gen'

In [33]:
var_df_dict = {}

for ind, file in enumerate(narr_final_dfs):
    var_df = pd.read_excel(NARR_FINAL_DATA_VAR_AGG_FOLDER + file)
    print(var_df.shape, file)
    var_df.sort_values([PLANT_CODE, YEAR, MONTH], ascending=[True, True, True], inplace=True)
    print(var_df.shape, file)
    var_df.reset_index(inplace=True, drop=True)
    var_df_dict[file.split('.')[0]] = var_df
    if ind > 0:
        print(list(var_df_dict[var_names[0]][PLANT_CODE]) == list(var_df[PLANT_CODE]))
        print(list(var_df_dict[var_names[0]][YEAR]) == list(var_df[YEAR]))
        print(list(var_df_dict[var_names[0]][MONTH]) == list(var_df[MONTH]))
    print(file.split('.')[0], '########')
    

(306576, 6) acpcp.agg.2011.2022.xlsx
(306576, 6) acpcp.agg.2011.2022.xlsx
acpcp ########
(306576, 6) air.agg.2011.2022.xlsx
(306576, 6) air.agg.2011.2022.xlsx
True
True
True
air ########
(306576, 6) albedo.agg.2011.2022.xlsx
(306576, 6) albedo.agg.2011.2022.xlsx
True
True
True
albedo ########
(306576, 6) apcp.agg.2011.2022.xlsx
(306576, 6) apcp.agg.2011.2022.xlsx
True
True
True
apcp ########
(306576, 6) dpt.agg.2011.2022.xlsx
(306576, 6) dpt.agg.2011.2022.xlsx
True
True
True
dpt ########
(306576, 6) pottmp.agg.2011.2022.xlsx
(306576, 6) pottmp.agg.2011.2022.xlsx
True
True
True
pottmp ########
(306576, 6) rhum.agg.2011.2022.xlsx
(306576, 6) rhum.agg.2011.2022.xlsx
True
True
True
rhum ########
(306576, 6) tcdc.agg.2011.2022.xlsx
(306576, 6) tcdc.agg.2011.2022.xlsx
True
True
True
tcdc ########
(306576, 6) uwnd.agg.2011.2022.xlsx
(306576, 6) uwnd.agg.2011.2022.xlsx
True
True
True
uwnd ########
(306576, 6) vwnd.agg.2011.2022.xlsx
(306576, 6) vwnd.agg.2011.2022.xlsx
True
True
True
vwnd #####

ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [50]:
narr_master_df = var_df_dict[var_names[0]].copy()
print(narr_master_df.shape, narr_master_df.columns)
for var_ in var_names[1:]:
    for col in [col for col in var_df_dict[var_].columns if var_ in col]:
        narr_master_df[col] = var_df_dict[var_][col]
        print(narr_master_df.shape, narr_master_df.columns)
    print(var_, '################')
print(narr_master_df.shape, narr_master_df.columns)
narr_master_df.to_excel('narr_master_df.xlsx')

(306576, 6) Index(['plant_code', 'year', 'month', 'acpcp_min', 'acpcp_max', 'acpcp_avg'], dtype='object')
(306576, 7) Index(['plant_code', 'year', 'month', 'acpcp_min', 'acpcp_max', 'acpcp_avg',
       'air_min'],
      dtype='object')
(306576, 8) Index(['plant_code', 'year', 'month', 'acpcp_min', 'acpcp_max', 'acpcp_avg',
       'air_min', 'air_max'],
      dtype='object')
(306576, 9) Index(['plant_code', 'year', 'month', 'acpcp_min', 'acpcp_max', 'acpcp_avg',
       'air_min', 'air_max', 'air_avg'],
      dtype='object')
air ################
(306576, 10) Index(['plant_code', 'year', 'month', 'acpcp_min', 'acpcp_max', 'acpcp_avg',
       'air_min', 'air_max', 'air_avg', 'albedo_min'],
      dtype='object')
(306576, 11) Index(['plant_code', 'year', 'month', 'acpcp_min', 'acpcp_max', 'acpcp_avg',
       'air_min', 'air_max', 'air_avg', 'albedo_min', 'albedo_max'],
      dtype='object')
(306576, 12) Index(['plant_code', 'year', 'month', 'acpcp_min', 'acpcp_max', 'acpcp_avg',
       'air_

In [52]:
narr_master_df.to_excel(NARR_FINAL_DATA_VAR_AGG_FOLDER + 'narr_master_df.xlsx')
narr_master_df

,plant_code,year,month,acpcp_min,acpcp_max,acpcp_avg,air_min,air_max,air_avg,albedo_min,...,rhum_avg,tcdc_min,tcdc_max,tcdc_avg,uwnd_min,uwnd_max,uwnd_avg,vwnd_min,vwnd_max,vwnd_avg
0,2,2011,1,0.0,17.757812,0.611873,269.307343,288.326172,277.948092,10.0,...,77.167627,0.625,95.750,45.600806,0.147941,4.579306,1.831262,0.268705,4.993980,2.192999
1,2,2011,2,0.0,8.242188,0.526839,273.286682,294.365631,283.260946,10.0,...,76.704510,0.000,99.750,35.709821,0.178692,4.874641,1.748523,0.199846,7.026369,2.905407
2,2,2011,3,0.0,23.429688,2.082684,281.186005,294.162506,288.654109,10.0,...,77.611954,0.000,96.375,38.161290,0.017701,5.251187,1.819689,0.267129,6.170527,3.009324
3,2,2011,4,0.0,40.454475,5.850930,283.376892,297.169800,291.931956,10.0,...,76.982448,0.000,75.625,25.220833,0.065307,6.336984,1.506685,0.146513,10.212384,3.824454
4,2,2011,5,0.0,18.414062,0.967468,285.724548,300.778992,294.387196,20.0,...,77.146185,0.000,83.875,25.858871,0.231988,3.841439,1.378504,0.202084,5.578331,2.654214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306571,58080,2022,8,0.0,44.875000,3.307262,299.087219,312.938995,307.784638,20.0,...,52.299591,1.250,74.000,29.947581,0.038513,4.308990,1.490517,0.254337,5.731156,2.833076
306572,58080,2022,9,0.0,5.406250,0.363281,297.405670,305.133575,302.037154,20.0,...,49.845902,0.000,51.000,10.329167,0.001900,3.845873,1.302594,0.001002,5.776958,2.720011
306573,58080,2022,10,0.0,20.023438,1.114903,283.826050,302.804230,294.231754,20.0,...,46.713495,0.000,91.125,27.862903,0.061705,4.966507,1.562787,0.176200,9.956518,3.295037
306574,58080,2022,11,0.0,11.468750,0.882181,276.443909,296.862946,285.472859,20.0,...,71.779675,0.000,98.000,42.762500,0.034166,4.416965,1.655933,0.169706,8.342846,3.361691


In [60]:
narr_plant_codes_list = narr_master_df[PLANT_CODE]
narr_plant_codes_list = np.unique(narr_plant_codes_list)
print(len(narr_plant_codes_list))

2129


#### 3. read [eia923_master_df] & [eia860_master_cap_df.xlsx]

In [154]:
eia_master_cap_df = pd.read_excel(
    EIA_FILTER_DATA_STORE_PATH + 'eia860_master_cap_df.xlsx')    

eia_master_gen_df = pd.read_excel(
    EIA_FILTER_DATA_STORE_PATH + 'eia923_master_df.xlsx')    

eia_master_cap_df = eia_master_cap_df[eia_master_cap_df[PLANT_CODE].isin(
    narr_plant_codes_list)]
eia_master_gen_df = eia_master_gen_df[eia_master_gen_df[PLANT_CODE].isin(
    narr_plant_codes_list)]
print(eia_master_cap_df.shape, eia_master_gen_df.shape)

(3514, 5) (306576, 4)


In [155]:
eia_master_cap_df

,plant_code,month,year,energy_source_1,nameplate_capacity_mw
0,2,7,1963,WAT,45.0
1,4,7,1967,WAT,75.0
2,4,8,1967,WAT,150.0
3,11,6,1966,WAT,72.9
4,12,8,1968,WAT,40.0
...,...,...,...,...,...
3509,6422,3,2022,WAT,4.1
3510,50654,7,2022,SUN,2.0
3511,54111,11,2022,GEO,42.5
3512,56336,2,2022,WND,145.0


In [156]:
eia_master_gen_df.sort_values(
    [PLANT_CODE, YEAR, MONTH], ascending=[True, True, True], inplace=True)
print(eia_master_gen_df.shape)
eia_master_gen_df.reset_index(inplace=True, drop=True)
eia_master_gen_df

(306576, 4)


,plant_code,month,year,net_gen
0,2,1,2011,11312.437
1,2,2,2011,9090.908
2,2,3,2011,22446.993
3,2,4,2011,20390.027
4,2,5,2011,7243.627
...,...,...,...,...
306571,58080,8,2022,23394.840
306572,58080,9,2022,21587.098
306573,58080,10,2022,28148.298
306574,58080,11,2022,34128.740


In [157]:
def get_cap_col(r):
    nameplate_cap = 0.0
    print(PLANT_CODE, r[PLANT_CODE], r[YEAR], r[MONTH])
    plant = r[PLANT_CODE]
    cap_df = eia_master_cap_df[eia_master_cap_df[PLANT_CODE] == plant]
    r[ENERGY_SOURCE] = list(cap_df[ENERGY_SOURCE])[0]
    cap_df_2 = cap_df[(cap_df[YEAR] < r[YEAR])]
    print(cap_df_2.shape, cap_df.shape)
    if cap_df_2.shape[0] == 0:
        if cap_df[(cap_df[YEAR] == r[YEAR])].shape[0] == 0:
            r[ENERGY_SOURCE] = None
            r[NAMEPLATE_CAPACITY] = None
            return r
        else:
            if cap_df[(cap_df[YEAR] == r[YEAR]) & (cap_df[MONTH] < r[MONTH])].shape[0] > 0:
                nameplate_cap = nameplate_cap + sum(list(cap_df[(
                    cap_df[YEAR] == r[YEAR]) & (cap_df[MONTH] < r[MONTH])][NAMEPLATE_CAPACITY]))

    nameplate_cap = nameplate_cap + sum(list(cap_df_2[NAMEPLATE_CAPACITY]))
    print(cap_df[(cap_df[YEAR] == r[YEAR])].shape[0])
    if cap_df[(cap_df[YEAR] == r[YEAR])].shape[0] > 0:
        print(cap_df[(cap_df[YEAR] == r[YEAR])].shape[0])
        print(cap_df[(cap_df[YEAR] == r[YEAR]) & (cap_df[MONTH] < r[MONTH])].shape)
        if cap_df[(cap_df[YEAR] == r[YEAR]) & (cap_df[MONTH] < r[MONTH])].shape[0] > 0:
#             print(cap_df[(cap_df[YEAR] == r[YEAR]) & (cap_df[MONTH] < r[MONTH])].shape[0])
            nameplate_cap = nameplate_cap + sum(list(cap_df[(
                cap_df[YEAR] == r[YEAR]) & (cap_df[MONTH] < r[MONTH])][NAMEPLATE_CAPACITY]))
    print(nameplate_cap)
    r[NAMEPLATE_CAPACITY] = nameplate_cap
    return r


In [172]:
eia_master_gen_df = eia_master_gen_df.apply(get_cap_col, axis=1)


In [163]:
eia_master_gen_df.to_excel(
    EIA_FILTER_DATA_STORE_PATH + 'eia_master_df.xlsx')    


In [164]:
print(eia_master_gen_df.shape)
eia_master_gen_df.head()

(306576, 6)


,plant_code,month,year,net_gen,energy_source_1,nameplate_capacity_mw
0,2.0,1.0,2011.0,11312.437,WAT,45.0
1,2.0,2.0,2011.0,9090.908,WAT,45.0
2,2.0,3.0,2011.0,22446.993,WAT,45.0
3,2.0,4.0,2011.0,20390.027,WAT,45.0
4,2.0,5.0,2011.0,7243.627,WAT,45.0


#### 4. combine (3) to get [eia_master_df.xlsx]


In [165]:
print(narr_master_df.shape)
narr_master_df.head()

(306576, 33)


,plant_code,year,month,acpcp_min,acpcp_max,acpcp_avg,air_min,air_max,air_avg,albedo_min,...,rhum_avg,tcdc_min,tcdc_max,tcdc_avg,uwnd_min,uwnd_max,uwnd_avg,vwnd_min,vwnd_max,vwnd_avg
0,2,2011,1,0.0,17.757812,0.611873,269.307343,288.326172,277.948092,10.0,...,77.167627,0.625,95.750,45.600806,0.147941,4.579306,1.831262,0.268705,4.993980,2.192999
1,2,2011,2,0.0,8.242188,0.526839,273.286682,294.365631,283.260946,10.0,...,76.704510,0.000,99.750,35.709821,0.178692,4.874641,1.748523,0.199846,7.026369,2.905407
2,2,2011,3,0.0,23.429688,2.082684,281.186005,294.162506,288.654109,10.0,...,77.611954,0.000,96.375,38.161290,0.017701,5.251187,1.819689,0.267129,6.170527,3.009324
3,2,2011,4,0.0,40.454475,5.850930,283.376892,297.169800,291.931956,10.0,...,76.982448,0.000,75.625,25.220833,0.065307,6.336984,1.506685,0.146513,10.212384,3.824454
4,2,2011,5,0.0,18.414062,0.967468,285.724548,300.778992,294.387196,20.0,...,77.146185,0.000,83.875,25.858871,0.231988,3.841439,1.378504,0.202084,5.578331,2.654214


In [166]:
print(eia_master_gen_df.shape)
eia_master_gen_df.head()

(306576, 6)


,plant_code,month,year,net_gen,energy_source_1,nameplate_capacity_mw
0,2.0,1.0,2011.0,11312.437,WAT,45.0
1,2.0,2.0,2011.0,9090.908,WAT,45.0
2,2.0,3.0,2011.0,22446.993,WAT,45.0
3,2.0,4.0,2011.0,20390.027,WAT,45.0
4,2.0,5.0,2011.0,7243.627,WAT,45.0


#### 5. master_df -> combine [narr_master_df] & [eia_master_df]

In [167]:
print(list(eia_master_gen_df[PLANT_CODE]) == list(narr_master_df[PLANT_CODE]))
print(list(eia_master_gen_df[YEAR]) == list(narr_master_df[YEAR]))
print(list(eia_master_gen_df[MONTH]) == list(narr_master_df[MONTH]))


True
True
True


In [169]:
narr_master_df[ENERGY_SOURCE] = eia_master_gen_df[ENERGY_SOURCE]
narr_master_df[NAMEPLATE_CAPACITY] = eia_master_gen_df[NAMEPLATE_CAPACITY]
narr_master_df[GENERATION] = eia_master_gen_df[GENERATION]


In [170]:
print(narr_master_df.shape)
narr_master_df.head()

(306576, 36)


,plant_code,year,month,acpcp_min,acpcp_max,acpcp_avg,air_min,air_max,air_avg,albedo_min,...,tcdc_avg,uwnd_min,uwnd_max,uwnd_avg,vwnd_min,vwnd_max,vwnd_avg,energy_source_1,nameplate_capacity_mw,net_gen
0,2,2011,1,0.0,17.757812,0.611873,269.307343,288.326172,277.948092,10.0,...,45.600806,0.147941,4.579306,1.831262,0.268705,4.993980,2.192999,WAT,45.0,11312.437
1,2,2011,2,0.0,8.242188,0.526839,273.286682,294.365631,283.260946,10.0,...,35.709821,0.178692,4.874641,1.748523,0.199846,7.026369,2.905407,WAT,45.0,9090.908
2,2,2011,3,0.0,23.429688,2.082684,281.186005,294.162506,288.654109,10.0,...,38.161290,0.017701,5.251187,1.819689,0.267129,6.170527,3.009324,WAT,45.0,22446.993
3,2,2011,4,0.0,40.454475,5.850930,283.376892,297.169800,291.931956,10.0,...,25.220833,0.065307,6.336984,1.506685,0.146513,10.212384,3.824454,WAT,45.0,20390.027
4,2,2011,5,0.0,18.414062,0.967468,285.724548,300.778992,294.387196,20.0,...,25.858871,0.231988,3.841439,1.378504,0.202084,5.578331,2.654214,WAT,45.0,7243.627


In [173]:
narr_master_df.to_excel(DATA_FOLDER + 'raw_master_df.xlsx', index=False)